In [1]:
%matplotlib inline
from pylab import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, graphviz_layout
from pyomo.environ import ConcreteModel, SolverFactory, TerminationCondition, \
    value, Var, Constraint, Expression, Objective, TransformationFactory, units as pyunits
from pyomo.network import Arc, SequentialDecomposition
from idaes.core.util.model_statistics import degrees_of_freedom
from pyomo.environ import (
    Block, Constraint, Expression, Var, Param, NonNegativeReals, units as pyunits)
from idaes.core.util.exceptions import ConfigurationError
from split_test2 import Separator1

from mixer_example import Mixer1

### TODO: CREATE INTRO

##### Import WaterTAP3 Package

In [2]:
import watertap as wt

In [3]:
import case_study_trains

##### Step 1: Look up unit process library. Returns a List.

##### Step 3: Set up IDAES flowsheet for watertap. This selects a property package for you (see water_props.py). The property package could be changed in the future for performing heat/energy and other balances.

##### Step 3: Add a water source and define inlet flow.

In [4]:
case_study_trains.case_study = "Carlsbad"
case_study_trains.reference = "NAWI"
case_study_trains.water_type = "Seawater"

In [5]:
m = wt.watertap_setup(dynamic = False)

In [6]:
m = wt.case_study_trains.get_case_study(flow = 4.5833, m = m)

/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)


In [7]:
wt.display.show_train2(model_name=m)

In [8]:
##### Step 5:  Run model and display results. The resutls display will be put into easier format to read for user soon.

In [9]:
import financials
financials.get_system_costing(m.fs)

In [10]:
result = wt.run_water_tap(m)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      660
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       65

Total number of variables............................:      260
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       48
                     variables with only upper bounds:        0
Tot

In [11]:
cost_variables = [
    "total_up_cost",
    "fixed_cap_inv_unadjusted",
    "fixed_cap_inv",
    "total_fixed_op_cost",
    "total_cap_investment",
    "land_cost",
    "working_cap",
    "catalysts_chemicals",#
    #"electricity",##
    "cap_replacement_parts", #
    "salaries",
    "benefits",
    "maintenance",
    "lab",
    "insurance_taxes",
    "base_employee_salary_cost",
    
    "cat_and_chem_cost",##
    "labor_and_other_fixed",#
    "consumer_price_index",#
    
    "electricity_cost", ##
    "other_var_cost"   ## 
]

In [12]:
variables = cost_variables + ["inlet_flow", "outlet_flow", "waste_flow"]

In [13]:
import case_study_trains
unit_process_names = case_study_trains.get_unit_processes(case_study_trains.case_study)

In [14]:
def get_results_table(model, unit_process_names, scenario = None, case_study = None):
    # could make a dictionary if betteR?
    value_list = [] 
    up_list = []
    scenario_list = []
    unit_list = []
    variable_list = []
    case_study_list = []
    
    for node in unit_process_names:
        
        for variable in variables:
            
            if variable == "inlet_flow":
                value_list.append(getattr(m.fs, node).inlet.flow_vol.extract_values()[0])
                variable_list.append(variable)
                unit_list.append("m3/s")
            elif variable == "outlet_flow":
                value_list.append(getattr(m.fs, node).outlet.flow_vol.extract_values()[0])
                variable_list.append(variable)
                unit_list.append("m3/s")
            elif variable == "waste_flow":
                value_list.append(getattr(m.fs, node).waste.flow_vol.extract_values()[0])
                variable_list.append(variable)
                unit_list.append("m3/s")
            else:
                # call m.fs.'node'.costing.'variable'() for each node and variable
                costing_attr = getattr(m.fs, node).costing
                cost_method = getattr(costing_attr, variable) #m.fs.swoi.costing.total_up_cost            
            
                if type(cost_method) == int:
                    cost_method = np.float64(cost_method)

                if type(cost_method) == np.float64:
                    cost_value = cost_method
                else:
                    cost_value = cost_method()
                # append ^ to a list
                value_list.append(cost_value)            
                variable_list.append(variable)
                unit_list.append("$MM")        
        
            up_list.append(node)
            scenario_list.append(scenario)
            case_study_list.append(case_study)
    
    df = pd.DataFrame()
    df["Case_Study"] = case_study_list
    df["Scenario"] = scenario_list
    df["Unit_Process"] = up_list
    df["Variable"] = variable_list
    df["Unit"] = unit_list
    df["Value"] = value_list
    
    #print(df)
    return df

In [15]:
t1 = get_results_table(m, unit_process_names, scenario = "Baseline", case_study = "Carlsbad")

In [16]:
t1

,Case_Study,Scenario,Unit_Process,Variable,Unit,Value
0,Carlsbad,Baseline,sw_onshore_intake,total_up_cost,$MM,2.237328e+01
1,Carlsbad,Baseline,sw_onshore_intake,fixed_cap_inv_unadjusted,$MM,2.030871e+01
2,Carlsbad,Baseline,sw_onshore_intake,fixed_cap_inv,$MM,2.098023e+01
3,Carlsbad,Baseline,sw_onshore_intake,total_fixed_op_cost,$MM,3.125703e-01
4,Carlsbad,Baseline,sw_onshore_intake,total_cap_investment,$MM,2.206071e+01
...,...,...,...,...,...,...
363,Carlsbad,Baseline,landfill,electricity_cost,$MM,0.000000e+00
364,Carlsbad,Baseline,landfill,other_var_cost,$MM,0.000000e+00
365,Carlsbad,Baseline,landfill,inlet_flow,m3/s,2.532148e-02
366,Carlsbad,Baseline,landfill,outlet_flow,m3/s,2.532123e-02


In [17]:
t1[t1.Variable == "total_up_cost"]

,Case_Study,Scenario,Unit_Process,Variable,Unit,Value
0,Carlsbad,Baseline,sw_onshore_intake,total_up_cost,$MM,22.373278
23,Carlsbad,Baseline,coag_and_floc,total_up_cost,$MM,13.654145
46,Carlsbad,Baseline,tri_media_filtration,total_up_cost,$MM,17.842173
69,Carlsbad,Baseline,sulfuric_acid_addition,total_up_cost,$MM,0.696074
92,Carlsbad,Baseline,sodium_bisulfite_addition,total_up_cost,$MM,1.196756
115,Carlsbad,Baseline,cartridge_filtration,total_up_cost,$MM,12.784471
138,Carlsbad,Baseline,ro_deep,total_up_cost,$MM,316.941264
161,Carlsbad,Baseline,lime_softening,total_up_cost,$MM,57.306799
184,Carlsbad,Baseline,co2_addition,total_up_cost,$MM,8.388663
207,Carlsbad,Baseline,chlorination_twb,total_up_cost,$MM,10.074838


In [18]:
t1.to_csv("results_v2.csv")

In [19]:
m.fs.costing.capital_investment_total.display()

capital_investment_total : Size=1
    Key  : Value
    None : 479.444813142111


In [ ]:
m.fs.costing.capital_investment_total.display()

In [20]:
m.fs.costing.LCOW.display()

LCOW : Size=1
    Key  : Value
    None : 1.4843802324562022e-06


In [ ]:
cost_range_list = []; #results will be inputted in this array
#up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.

#for value_change in pct_to_target1: # cycles through each value from MC range
for value_change in [0.4, 0.8]: #, 0.9]:

    # create and build model
    m = wt.watertap_setup(dynamic = False)
    m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

    m.fs.tri_media_filtration.water_recovery.fix(value_change)

    # set variable to MC value
    wt.run_water_tap(m)
    results_table = get_results_table(m, unit_process_name)
    cost_range_list.append(results_table.total_up_cost.sum())


In [ ]:
cost_range_list

In [ ]:
import time
from multiprocessing import Pool
import multiprocessing

mu = 0.6
sigma = .1
num_reps = 50

input_list = np.random.normal(mu,sigma, size = num_reps) #, sigma, num_reps).round(4)

count, bins, ignored = plt.hist(input_list, 25, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
          linewidth=2, color='r')
plt.show()

### INPUT TO MODEL LIST: ### CAN BE AUTOMATED FOR USER TO LABEL THE VARIABLE. TOOD ###
no_of_proc = 4
list_final = []
for i in range(no_of_proc):
    part2 = len(input_list) / no_of_proc
    i2 = ((i+1)*part2)
    list1 = input_list[int(i*part2):int(i2)]
    list_final.append(list1)
    
    
def monte_run(list_final):
    print('goes in')

    up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.
    cost_range_list = []; #results will be inputted in this array

    #for value_change in pct_to_target1: # cycles through each value from MC range
    for value_change in list_final:

        # create and build model
        m = wt.watertap_setup(dynamic = False)
        m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

        getattr(m.fs, up_name).water_recovery.fix(value_change)

        # set variable to MC value
        result = wt.run_water_tap(m)
        results_table = get_results_table(m, unit_process_names)
        cost_range_list.append(results_table.total_up_cost.sum())


    return cost_range_list

startTime = time.time()

pool=Pool()
dfs = pool.map(monte_run, list_final) #SomeClass().preprocess_data()

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
####TO DO LOAD AND SAVE!!

In [ ]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [ ]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [ ]:
# wt.display.show_train(TT)